google-cloud-bigquery (Client Library): This is the primary library for interacting with BigQuery from Python. It lets you create clients, run SQL queries, manage datasets/tables, and load data. This is what you'll use to execute the SQL commands from your outline.

google-genai (GenAI SDK): This SDK is used for direct interaction with the Gemini API for tasks like text generation. However, for the RAG flow you described, where the vector search and initial LLM calls are done within BigQuery ML (BQML), you will primarily use the google-cloud-bigquery library to run the BQML SQL functions. The final chatbot component could use google-genai for the conversational part, but keeping the entire RAG pipeline in BQML via the BigQuery client is often more efficient.

pandas: Essential for data manipulation and viewing query results returned from BigQuery.

google-cloud-bigquery-storage: Recommended dependency for faster data downloads.

In [2]:
!pip install google-cloud-bigquery pandas google-cloud-bigquery-storage

In [1]:
import os
from google.cloud import bigquery
from google.cloud.bigquery import LoadJobConfig, SourceFormat
from IPython.display import display

In [3]:
# --- Configuration ---
PROJECT_ID = "qwiklabs-gcp-01-c75658565206"  # <-- **Replace this**
DATASET_ID = "aurora_rag_data"
TABLE_ID = "aurora_bay_faqs"
GCS_URI = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"
LOCATION = "US"  # BigQuery dataset location (e.g., 'US' or 'EU')

# taken from UI
# Service account id: bqcx-720196750972-tcql@gcp-sa-bigquery-condel.iam.gserviceaccount.com
CONNECTION_ID = "embedding_conn" # manually created in bigquery UI

# manually grant IAM permissions for embedding_conn

# Initialize the BigQuery Client
bq_client = bigquery.Client(project=PROJECT_ID)

print(f"Project: {PROJECT_ID}")
print(f"Dataset: {DATASET_ID}")

# add IAM permissions manually via UI

Project: qwiklabs-gcp-01-c75658565206
Dataset: aurora_rag_data


In [4]:
dataset_ref = bq_client.dataset(DATASET_ID, project=PROJECT_ID)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = LOCATION

try:
    dataset = bq_client.create_dataset(dataset, timeout=30)
    print(f"Created dataset {PROJECT_ID}.{DATASET_ID}")
except Exception as e:
    # Dataset likely already exists, which is fine
    if 'Already Exists' in str(e):
        print(f"Dataset {PROJECT_ID}.{DATASET_ID} already exists.")
    else:
        raise

Created dataset qwiklabs-gcp-01-c75658565206.aurora_rag_data


In [5]:
# Define the schema based on the file content
schema = [
    bigquery.SchemaField("question", "STRING"),
    bigquery.SchemaField("answer", "STRING"),
]

job_config = LoadJobConfig(
    source_format=SourceFormat.CSV,
    skip_leading_rows=1,  # Assuming a header row
    autodetect=True,     # You can use autodetect for simplicity, or define the full schema
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE, # Overwrite table if it exists
)

table_id_full = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"
load_job = bq_client.load_table_from_uri(
    GCS_URI,
    table_id_full,
    job_config=job_config,
)

print(f"Starting load job {load_job.job_id}...")
load_job.result() # Wait for the job to complete

print(f"Loaded {load_job.output_rows} rows into {table_id_full}")

# # test validation of load job
# # Display the first few rows to verify
# query = f"SELECT * FROM `{table_id_full}` LIMIT 5"
# df_source = bq_client.query(query).to_dataframe()
# print("\n--- Source Table Preview ---")
# display(df_source)

Starting load job 7c493462-63e9-4f0e-b9ad-4f768bad24a3...
Loaded 50 rows into qwiklabs-gcp-01-c75658565206.aurora_rag_data.aurora_bay_faqs

--- Source Table Preview ---


,string_field_0,string_field_1
0,When was Aurora Bay founded?,Aurora Bay was founded in 1901 by a group of f...
1,What is the population of Aurora Bay?,Aurora Bay has a population of approximately 3...
2,Where is the Aurora Bay Town Hall located?,The Town Hall is located at 100 Harbor View Ro...
3,Who is the current mayor of Aurora Bay?,"The current mayor is Linda Greenwood, elected ..."
4,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...
